In [ ]:
# To-Do
#1) transfer files locally from lxplus or maybe figure out venv on lxplus
#2) CM of phi for image?
#3) bb-center a la photography talk
#4) slide 9: remake these for diHiggs events that have the Higgs back-to-back ie no ISR. Are the blobs more pronounced than non-back-to-back diHiggs events?
#5) increase pixels to see if better?
#6) compare expected yields post-selection from this to other approaches. Much higher signal stats here! But by how much?
#7) subtract average a la Anna's idea
#8) store images by jet category?
# ...
# ...
#9) script image making --> different pixel sizes, jet categories
#10) compare ROC curves for different approaches a la Evan's work

In [ ]:
# Import the needed libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import os 

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2, l1
from keras.initializers import Constant
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


# Fix random seed for reproducibility
seed = 7
np.random.seed(seed)

import sys
sys.path.insert(0, '/home/btannenw/Desktop/ML/dihiggsMLProject/')
from utils.commonFunctions import *

In [ ]:
# Get image data
hh_h5  = h5.File('/home/btannenw/Desktop/ML/dihiggsMLProject/convolutionalNN/pp2hh4b_1M_20files_composite_pix15_phiCM_cat/images/pp2hh4b_1M_20files_composite_pix15_phiCM_cat_allImages.h5', 'r')
qcd_h5 = h5.File('/home/btannenw/Desktop/ML/dihiggsMLProject/convolutionalNN/ppTo4b_4M_20files_composite_pix15_phiCM_cat/images/ppTo4b_4M_20files_composite_pix15_phiCM_cat_allImages.h5', 'r')

In [ ]:
# Make composite dihiggs images
#hh_combImages = []
##for iEvt in range(0, len(hh_imgs['trackImgs'])):
#for iEvt in range(0, 5000):
#    if iEvt%500 == 0:
#        print('Processed {} dihiggs events'.format(iEvt) )
#    hh_combImages.append( np.stack( (hh_imgs['trackImgs'][iEvt], hh_imgs['nHadronImgs'][iEvt], hh_imgs['photonImgs'][iEvt]), axis=-1) )
#print(len(hh_combImages))

#hh = np.stack( (hh_combImages), axis=0)

In [ ]:
hh_h5.keys()

In [ ]:
# Read data
#nEvents = 49757
#hh = hh_h5['compositeImgs'][:nEvents]
#qcd = qcd_h5['compositeImgs'][:nEvents]

hh = hh_h5['compositeImgs']
qcd = qcd_h5['compositeImgs']
print(len(hh), len(qcd))
#hh = list(hh)
#qcd = list(qcd)

In [ ]:
#nEvents_4j = 19000
#hh_4j = hh_h5['compositeImgs_4j'][:nEvents_4j]
#qcd_4j = qcd_h5['compositeImgs_4j'][:nEvents_4j]

hh_4j = hh_h5['compositeImgs_<4j']
qcd_4j = qcd_h5['compositeImgs_<4j']
hh_4j0b = hh_h5['compositeImgs_>=4j0b']
qcd_4j0b = qcd_h5['compositeImgs_>=4j0b']
hh_4j1b = hh_h5['compositeImgs_>=4j1b']
qcd_4j1b = qcd_h5['compositeImgs_>=4j1b']

hh_4j2b = hh_h5['compositeImgs_>=4j2b']
qcd_4j2b = qcd_h5['compositeImgs_>=4j2b']
hh_4j3b = hh_h5['compositeImgs_>=4j3b']
qcd_4j3b = qcd_h5['compositeImgs_>=4j3b']
hh_4j4b = hh_h5['compositeImgs_>=4j4b']
qcd_4j4b = qcd_h5['compositeImgs_>=4j4b']

hh_g4j4b = hh_h5['compositeImgs_>=4j>=4b']
qcd_g4j4b = qcd_h5['compositeImgs_>=4j>=4b']

#hh_4j
#hh  = np.asarray( [ img for img, njets in zip(hh_h5['compositeImgs'], hh_h5['nJets']) if njets >= 4 ] )
#qcd  = np.asarray( [ img for img, njets in zip(qcd_h5['compositeImgs'], qcd_h5['nJets']) if njets >= 4 ] )
#hh  = np.asarray( [ img for img in hh_h5['compositeImgs'] ] )
#qcd  = np.asarray( [ img for img in qcd_h5['compositeImgs'] ] )

hh_all = list(hh_4j) + list(hh_4j0b) + list(hh_4j1b) + list(hh_4j2b) + list(hh_4j3b) + list(hh_4j4b) + list(hh_g4j4b)

#hh = []
#nEvts = len(hh_h5['nJets'])
#for iEvt in range(0, nEvts):
#    if (100*(iEvt+1)/nEvts)%10 == 0:
#        print( 'Processed {}% hh...'.format( (100*(iEvt+1))/nEvts))
#    if hh_h5['nJets'][iEvt]>3:
#        hh.append(hh_h5['compositeImgs'][iEvt])

print(len(hh), len(qcd), len(hh_4j)+len(hh_4j0b)+len(hh_4j1b)+len(hh_4j2b)+len(hh_4j3b)+len(hh_4j4b)+len(hh_g4j4b), len(hh_all))

In [ ]:
#nEvents = min(len(hh), len(qcd))
#nEvents = min(len(hh_imgs['compositeImgs']), len(qcd_imgs['compositeImgs']))

# Make labels
hh_labels = np.ones( len(hh) )
qcd_labels = np.zeros( len(qcd) )

# Make combined dataset
#all_images = np.concatenate ( (hh.copy(), qcd.copy()), axis=0)
all_images = np.concatenate ( (hh, qcd) )
all_labels = np.concatenate ( (hh_labels.copy(), qcd_labels.copy()), axis=0)
print(all_images.shape, all_labels.shape)

#all_images, all_labels = shuffle(all_images, all_labels, random_state=0)
imgs_train, imgs_test, labels_train, labels_test = train_test_split(all_images, all_labels, test_size=0.25, shuffle= True, random_state=30)

In [ ]:
# Layer options
l2_reg = tf.keras.regularizers.l2(1e-4)
conv_kwargs = dict(
            activation="relu",
            #kernel_initializer=tf.keras.initializers.lecun_normal(),
            #kernel_regularizer=l2_reg,
    )

dense_kwargs = conv_kwargs

# bias options
initial_bias = np.log([len(hh)/len(qcd)])
output_bias = Constant(initial_bias)
print("initial bias: {}".format(initial_bias))

# class weights
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
total = len(qcd) + len(hh)
weight_for_0 = (1 / len(qcd))*(total)/2.0 
weight_for_1 = (1 / len(hh))*(total)/2.0
class_weight = {0: weight_for_0, 1: weight_for_1}
print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

In [ ]:
def evaluateModel( _model, _history, _data, _labels):
    # *** A. Make prediction data
    hh_data_test    = np.asarray([x for x,y in zip(_data, _labels) if y==1])
    hh_labels_test  = np.asarray([y for x,y in zip(_data, _labels) if y==1])
    qcd_data_test   = np.asarray([x for x,y in zip(_data, _labels) if y==0])
    qcd_labels_test = np.asarray([y for x,y in zip(_data, _labels) if y==0])

    # *** B. Load model 

    # *** C. Make history plots
    print("*** Training History Plots")
    makeHistoryPlots( _history, ['accuracy', 'loss', 'auc'] )

    # *** C. Make predictions
    score_hh = _model.evaluate(hh_data_test, hh_labels_test)
    score_qcd = _model.evaluate(qcd_data_test, qcd_labels_test)
    print(score_hh, score_qcd)
    pred_hh = _model.predict(hh_data_test)
    pred_qcd = _model.predict(qcd_data_test)
    
    # *** D. make output score plot
    print("*** Output Score Plot")
    _nBins = 40
    predictionResults = {'hh_pred':pred_hh, 'qcd_pred':pred_qcd}
    compareManyHistograms( predictionResults, ['hh_pred', 'qcd_pred'], 2, 'Signal Prediction', 'CNN Score', 0, 1, _nBins, _yMax = 5, _normed=True, savePlot=False )

    # *** E. Get best cut value for CNN assuming some minimal amount of signal
    pred_hh_sig = [x[0] for x in pred_hh.copy()]
    pred_qcd_sig = [x[0] for x in pred_qcd.copy()]

    sig, cut, sigErr = returnBestCutValue('CNN', pred_hh_sig, pred_qcd_sig, _minBackground=400e3, _testingFraction=0.025)
    
    # *** F. Confusion matrix
    print("*** Confusion Matrix")
    preds_test = _model.predict(_data)
    cm = confusion_matrix( _labels, preds_test > cut)
    print(cm)
    print('QCD called QCD (True Negatives): {} ({}%)'.format( cm[0][0], round(100*(cm[0][0]/sum(cm[0]))) ))
    print('QCD called Dihiggs (False Positives):  {} ({}%)'.format( cm[0][1], round(100*(cm[0][1]/sum(cm[0]))) ))
    print('Dihiggs called QCD (False Negatives):  {} ({}%)'.format( cm[1][0], round(100*(cm[1][0]/sum(cm[1]))) ))
    print('Dihiggs called Dihiggs (True Positives):  {} ({}%)'.format( cm[1][1], round(100*(cm[1][1]/sum(cm[1]))) ))
    print('Total Dihiggs: ', np.sum(cm[1]))
    
    # *** 7. Make ROC curve
    print("*** ROC Curve")
    makeEfficiencyCurves( dict(label="CNN", labels=_labels, prediction=preds_test, color="blue"), _modelName='CNN')
    #utils/commonFunctions.py:def makeEfficiencyCurves(*data, _modelName='', savePlot=False, saveDir=''):

In [ ]:
def makeModelCNN():
    # Define model
    _model = Sequential()

    # Convolutional part
    _model.add( Conv2D(16, (3, 3), input_shape=(15, 15, 3), **conv_kwargs) )
    _model.add( MaxPooling2D((2, 2)) )
    _model.add( Conv2D(32, (3, 3), **conv_kwargs))
    _model.add( MaxPooling2D((2, 2)))
    _model.add( Conv2D(32, (2, 2), **conv_kwargs))
    #model.add(layers.Conv2D(64, (3, 3), activation='relu'))

    # Feed-forward part
    _model.add( Flatten())
    _model.add( Dense(64, **dense_kwargs))
    _model.add( BatchNormalization() )
    _model.add( Dense(64, **dense_kwargs))
    #model.add( Dropout(0.2) )
    _model.add( Dense(1, activation='sigmoid', bias_initializer=output_bias) )
    
    _model.summary()
    
    metrics = [ #tf.keras.metrics.categorical_accuracy,
            'accuracy',
            tf.keras.metrics.AUC(name='auc'),
    ]

    _model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=metrics)
              #metrics=['accuracy'])
    
    return _model


In [ ]:
# *** 1. Define output directory
topDir = "composite_pixel25_3conv"
name = 'epochs15'
if not os.path.exists(topDir):
    os.makedirs(topDir)
model_dir = os.path.join(topDir, "", "models", name)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
# *** 2. Define callbacks for training
fit_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(model_dir, name)+'.hdf5',
            save_best_only=True,
            save_weights_only=True,
            #monitor="val_auc",
            #mode="max",
            monitor="val_loss",
            mode="min",
        ),
    tf.keras.callbacks.EarlyStopping(
            #monitor="val_auc",
            #mode="max",
            monitor='val_loss', 
            mode='min', 
            verbose=1, 
            patience=15,  
            min_delta=.0025,
        ),
    ]

In [ ]:
# *** 3A. Train without weights

modelNoWeights = makeModelCNN()

historyNoWeights = modelNoWeights.fit(imgs_train, labels_train, epochs=50, 
                    shuffle=True,
                    batch_size=512,
                    validation_data=(imgs_test, labels_test),
                    callbacks=fit_callbacks,
                    )

evaluateModel( modelNoWeights, historyNoWeights, imgs_test, labels_test)


In [ ]:
modelWeighted = makeModelCNN()

historyWeighted = modelWeighted.fit(imgs_train, labels_train, epochs=50, 
                    shuffle=True,
                    batch_size=512,
                    validation_data=(imgs_test, labels_test),
                    callbacks=fit_callbacks,
                    class_weight=class_weight
                   )

evaluateModel( modelWeighted, historyWeighted, imgs_test, labels_test)


In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.0, 1])
plt.legend(loc='lower right')
plt.show()

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0., 1])
plt.legend(loc='lower right')
plt.show()

plt.plot(history.history['auc'], label='AUC')
plt.plot(history.history['val_auc'], label = 'val_AUC')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.ylim([0., 1])
plt.legend(loc='lower right')
plt.show()

#test_loss, test_acc = model.evaluate(imgs_test,  labels_test, verbose=2)

In [ ]:
# *** 4B. Get signifiance for any user-specified NN score cut value
testingFraction = 0.1
lumiscale_hh  = getLumiScaleFactor(testingFraction, True, 25e3)
lumiscale_qcd = getLumiScaleFactor(testingFraction, False, 50e3)
cut = 0.48
_nSignal = sum( value > cut for value in pred_hh_sig)*lumiscale_hh
_nBackground = sum( value > cut for value in pred_qcd_sig)*lumiscale_qcd

print('nSig = {0} , nBkg = {1} with significance = {2} for NN score > {3}'.format(_nSignal, _nBackground, _nSignal/np.sqrt(_nBackground), cut) )